# atmodeller

## Tutorial 1: basic operation

Import the required packages and set the package logger to the INFO level. For more output you could instead set it to DEBUG.

In [1]:
from atmodeller import InteriorAtmosphereSystem, Molecule, SystemConstraint, OCEAN_MOLES, MolarMasses, logger
from atmodeller.thermodynamics import PeridotiteH2O, NoSolubility, BasaltDixonCO2

import logging

logger.setLevel(logging.DEBUG)

10:34:41 - atmodeller           - INFO      - atmodeller version 0.1.0


### 1. Simple H2O-H2 system with prescribed H2O pressure

We define a list of the molecules we wish to include in the interior-atmosphere system and how they partition between the melt and the atmosphere, and the solid and the melt:

In [2]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))

10:34:44 - atmodeller.core      - INFO      - Creating a molecule: H2O
10:34:44 - atmodeller.core      - DEBUG     - element count = 
{'H': 2, 'O': 1}
10:34:44 - atmodeller.core      - INFO      - Creating a molecule: H2
10:34:44 - atmodeller.core      - DEBUG     - element count = 
{'H': 2}


Although a choice is often made to constrain oxygen fugacity, we nevertheless need to explicitly include O2 as a molecule in the interior-atmosphere system:

In [3]:
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules

10:34:47 - atmodeller.core      - INFO      - Creating a molecule: O2
10:34:47 - atmodeller.core      - DEBUG     - element count = 
{'O': 2}


[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x11ea06ed0>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, formation_constants=(0.05817258823529415, -251.80119852941178), molar_mass=0.018015200000000002),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x111e28750>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, formation_constants=(0, 0), molar_mass=0.0020158),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x11f534ed0>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, formation_constants=(0, 0), molar_mass=0.0319988)]

We can then create an interior-atmosphere system using the list of molecules. Note that this creates a planet with 'default properties' (a reduced and molten Earth). Adjusting the planet properties will be covered in a later tutorial.

In [4]:
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules)

10:34:50 - atmodeller.core      - INFO      - Creating a new planet
10:34:50 - atmodeller.core      - INFO      - Mantle mass (kg) = 4.208261222595111e+24
10:34:50 - atmodeller.core      - INFO      - Mantle melt fraction = 1.0
10:34:50 - atmodeller.core      - INFO      - Core mass fraction = 0.295334691460966
10:34:50 - atmodeller.core      - INFO      - Planetary radius (m) = 6371000.0
10:34:50 - atmodeller.core      - INFO      - Planetary mass (kg) = 5.972e+24
10:34:50 - atmodeller.core      - INFO      - Surface temperature (K) = 2000.000000
10:34:50 - atmodeller.core      - INFO      - Surface gravity (m/s^2) = 9.819973426224687
10:34:50 - atmodeller.core      - INFO      - Oxygen fugacity model (mantle) = IronWustiteBufferOneill
10:34:50 - atmodeller.core      - INFO      - Oxygen fugacity log10 shift = 0.000000
10:34:50 - atmodeller.core      - INFO      - Creating a new interior-atmosphere system
10:34:50 - atmodeller.core      - INFO      - Molecules = ['H2', 'O2', 'H2O']
10

Notice that for this simple system it has identified the single reaction that relates the 3 chosen molecules. To solve the system, we provide a constraint of the H2O pressure in bar:

In [5]:
H2O_pressure: SystemConstraint = SystemConstraint(species='H2O', value=1, field='pressure')
constraints: list[SystemConstraint] = [H2O_pressure]

A second constraint is required to close the system of equations, and this is conveniently specified using `fo2_constraint=True`, which (without specifying additional arguments) will impose the oxygen fugacity at the Iron-Wustite buffer. Hence we can solve the interior-atmosphere system at a given temperature to determine the partial pressures of our chosen molecules:

In [6]:
interior_atmosphere.solve(constraints, fo2_constraint=True)

10:34:57 - atmodeller.core      - INFO      - Constraints: [SystemConstraint(species='H2O', value=1, field='pressure')]
10:34:57 - atmodeller.core      - INFO      - Pressure constraints only so attempting to solve a linear reaction network
10:34:57 - atmodeller.core      - INFO      - Solving the reaction network
10:34:57 - atmodeller.core      - INFO      - Adding fO2 as an additional constraint using IronWustiteBufferOneill with fO2_shift = 0.00
10:34:57 - atmodeller.core      - INFO      - The necessary number of constraints will be applied to the reaction network to solve the system
10:34:57 - atmodeller.core      - INFO      - Row 00: Reaction 0: 1.0 H2 + 0.5 O2 = 1.0 H2O
10:34:57 - atmodeller.core      - INFO      - Row 01: Setting H2O partial pressure
10:34:57 - atmodeller.core      - INFO      - Row 02: Setting O2 partial pressure
10:34:57 - atmodeller.core      - DEBUG     - Design matrix = 
[[-1.  -0.5  1. ]
 [ 0.   0.   1. ]
 [ 0.   1.   0. ]]
10:34:57 - atmodeller.core    

{'H2': 1.4663333811991948, 'O2': 3.9099946023954954e-08, 'H2O': 1.0}

You can access the solution directly using:

In [7]:
interior_atmosphere.pressures_dict

{'H2': 1.4663333811991948, 'O2': 3.9099946023954954e-08, 'H2O': 1.0}

### 2. System with C and H and prescribed pressures

We now extend the molecule list to additionally include C-species:

In [8]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
molecules

14:36:06 - atmodeller.core      - INFO      - Creating a molecule: H2O
14:36:06 - atmodeller.core      - INFO      - Creating a molecule: H2
14:36:06 - atmodeller.core      - INFO      - Creating a molecule: O2
14:36:06 - atmodeller.core      - INFO      - Creating a molecule: CO
14:36:06 - atmodeller.core      - INFO      - Creating a molecule: CO2


Element count: {'H': 2, 'O': 1}
Element count: {'H': 2}
Element count: {'O': 2}
Element count: {'C': 1, 'O': 1}
Element count: {'C': 1, 'O': 2}


[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x120a51d50>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, formation_constants=(0.05817258823529415, -251.80119852941178), molar_mass=0.018015200000000002),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x10b53ba50>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, formation_constants=(0, 0), molar_mass=0.0020158),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x11d0fca90>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, formation_constants=(0, 0), molar_mass=0.0319988),
 Molecule(name='CO', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x11e91b410>, solid_melt_distribution_coefficient=0, elements={'C': 1, 'O': 1}, element_masses={'C': 0.0120107, 'O': 0.0159994}, fo

In [9]:
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules)

14:36:11 - atmodeller.core      - INFO      - Creating a new planet
14:36:11 - atmodeller.core      - INFO      - Mantle mass (kg) = 4.208261222595111e+24
14:36:11 - atmodeller.core      - INFO      - Mantle melt fraction = 1.0
14:36:11 - atmodeller.core      - INFO      - Core mass fraction = 0.295334691460966
14:36:11 - atmodeller.core      - INFO      - Planetary radius (m) = 6371000.0
14:36:11 - atmodeller.core      - INFO      - Planetary mass (kg) = 5.972e+24
14:36:11 - atmodeller.core      - INFO      - Surface temperature (K) = 2000.000000
14:36:11 - atmodeller.core      - INFO      - Surface gravity (m/s^2) = 9.819973426224687
14:36:11 - atmodeller.core      - INFO      - Oxygen fugacity model (mantle) = IronWustiteBufferOneill
14:36:11 - atmodeller.core      - INFO      - Oxygen fugacity log10 shift = 0.000000
14:36:11 - atmodeller.core      - INFO      - Creating a new interior-atmosphere system
14:36:11 - atmodeller.core      - INFO      - Molecules = ['CO', 'H2', 'O2', 'CO

['H', 'C', 'O']


Note now the system has identified two reactions in the network. With C present in the system we must provide at least 2 constraints, in addition to the oxygen fugacity:

In [10]:
H2O_pressure: SystemConstraint = SystemConstraint(species='H2O', value=1, field='pressure')
CO2_pressure: SystemConstraint = SystemConstraint(species='CO2', value=1, field='pressure' )
constraints: list[SystemConstraint] = [H2O_pressure, CO2_pressure]

In [11]:
interior_atmosphere.solve(constraints, fo2_constraint=True)
interior_atmosphere.pressures_dict

14:36:20 - atmodeller.core      - INFO      - Constraints: [SystemConstraint(species='H2O', value=1, field='pressure'),
 SystemConstraint(species='CO2', value=1, field='pressure')]
14:36:20 - atmodeller.core      - INFO      - Pressure constraints only so attempting to solve a linear reaction network
14:36:20 - atmodeller.core      - INFO      - Solving the reaction network
14:36:20 - atmodeller.core      - INFO      - Adding fO2 as an additional constraint using IronWustiteBufferOneill with fO2_shift = 0.00
14:36:20 - atmodeller.core      - INFO      - The necessary number of constraints will be applied to the reaction network to solve the system
14:36:20 - atmodeller.core      - INFO      - Row 00: Reaction 0: 1.0 CO + 0.5 O2 = 1.0 CO2
14:36:20 - atmodeller.core      - INFO      - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
14:36:20 - atmodeller.core      - INFO      - Row 02: Setting H2O partial pressure
14:36:20 - atmodeller.core      - INFO      - Row 03: Setting CO2 partial pre

{'CO': 6.580988292698317,
 'H2': 1.4663333811991948,
 'O2': 3.9099946023954954e-08,
 'CO2': 1.0,
 'H2O': 1.0}

There is not a requirement to necessarily impose the oxygen fugacity as a constraint. Instead, we can simply impose three pressure constraints (that span the reaction set) and allow for the oxygen fugacity to be solved. Note that if we do not specify an appropriate range of constraints we cannot solve the system of equations to give a unique solution and hence the code will raise an exception relating to a singular matrix.

In [12]:
H2O_pressure: SystemConstraint = SystemConstraint(species='H2O', value=1, field='pressure')
H2_pressure: SystemConstraint = SystemConstraint(species='H2', value=1, field='pressure' )
CO_pressure: SystemConstraint = SystemConstraint(species='CO', value=1, field='pressure' )
constraints: list[SystemConstraint] = [H2O_pressure, H2_pressure, CO_pressure]

In [13]:
interior_atmosphere.solve(constraints, fo2_constraint=False)
interior_atmosphere.pressures_dict

14:36:54 - atmodeller.core      - INFO      - Constraints: [SystemConstraint(species='H2O', value=1, field='pressure'),
 SystemConstraint(species='H2', value=1, field='pressure'),
 SystemConstraint(species='CO', value=1, field='pressure')]
14:36:54 - atmodeller.core      - INFO      - Pressure constraints only so attempting to solve a linear reaction network
14:36:54 - atmodeller.core      - INFO      - Solving the reaction network
14:36:54 - atmodeller.core      - INFO      - The necessary number of constraints will be applied to the reaction network to solve the system
14:36:54 - atmodeller.core      - INFO      - Row 00: Reaction 0: 1.0 CO + 0.5 O2 = 1.0 CO2
14:36:54 - atmodeller.core      - INFO      - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
14:36:54 - atmodeller.core      - INFO      - Row 02: Setting H2O partial pressure
14:36:54 - atmodeller.core      - INFO      - Row 03: Setting H2 partial pressure
14:36:54 - atmodeller.core      - INFO      - Row 04: Setting CO partial 

{'CO': 1.0,
 'H2': 1.0,
 'O2': 8.407010711071814e-08,
 'CO2': 0.22281355261277533,
 'H2O': 1.0}

### 3. System with C and H and mixed constraints

A typical use case is to define an interior-atmosphere system with a combination of pressure and mass constraints. We define the same molecule set as before:

In [14]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
molecules

14:37:15 - atmodeller.core      - INFO      - Creating a molecule: H2O
14:37:15 - atmodeller.core      - INFO      - Creating a molecule: H2
14:37:15 - atmodeller.core      - INFO      - Creating a molecule: O2
14:37:15 - atmodeller.core      - INFO      - Creating a molecule: CO
14:37:15 - atmodeller.core      - INFO      - Creating a molecule: CO2


Element count: {'H': 2, 'O': 1}
Element count: {'H': 2}
Element count: {'O': 2}
Element count: {'C': 1, 'O': 1}
Element count: {'C': 1, 'O': 2}


[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x11e929090>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, formation_constants=(0.05817258823529415, -251.80119852941178), molar_mass=0.018015200000000002),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x11d87d4d0>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, formation_constants=(0, 0), molar_mass=0.0020158),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x120a52dd0>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, formation_constants=(0, 0), molar_mass=0.0319988),
 Molecule(name='CO', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x10b3f42d0>, solid_melt_distribution_coefficient=0, elements={'C': 1, 'O': 1}, element_masses={'C': 0.0120107, 'O': 0.0159994}, fo

Now we define the constraints, and in this case we want to constrain the total mass of C and H in the system that can partition between the various reservoirs.

In [16]:
number_of_earth_oceans: float = 1
# C/H ratio by mass.
ch_ratio: float = 1

mass_H: float = number_of_earth_oceans * OCEAN_MOLES * MolarMasses().H2
mass_C: float = ch_ratio * mass_H

constraints: list[SystemConstraint] = [
    SystemConstraint(species="H", value=mass_H, field="mass"),
    SystemConstraint(species="C", value=mass_C, field="mass"),
]

We need to apply one more constraint, which is usually the oxygen fugacity and this can be done directly as before by specifying `fo2_constraint=True`:

In [17]:
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules)
interior_atmosphere.solve(constraints, fo2_constraint=True)
interior_atmosphere.pressures_dict

14:37:22 - atmodeller.core      - INFO      - Creating a new planet
14:37:22 - atmodeller.core      - INFO      - Mantle mass (kg) = 4.208261222595111e+24
14:37:22 - atmodeller.core      - INFO      - Mantle melt fraction = 1.0
14:37:22 - atmodeller.core      - INFO      - Core mass fraction = 0.295334691460966
14:37:22 - atmodeller.core      - INFO      - Planetary radius (m) = 6371000.0
14:37:22 - atmodeller.core      - INFO      - Planetary mass (kg) = 5.972e+24
14:37:22 - atmodeller.core      - INFO      - Surface temperature (K) = 2000.000000
14:37:22 - atmodeller.core      - INFO      - Surface gravity (m/s^2) = 9.819973426224687
14:37:22 - atmodeller.core      - INFO      - Oxygen fugacity model (mantle) = IronWustiteBufferOneill
14:37:22 - atmodeller.core      - INFO      - Oxygen fugacity log10 shift = 0.000000
14:37:22 - atmodeller.core      - INFO      - Creating a new interior-atmosphere system
14:37:22 - atmodeller.core      - INFO      - Molecules = ['CO', 'H2', 'O2', 'CO

['H', 'C', 'O']


14:37:22 - atmodeller.core      - DEBUG     - residual_reaction = [ 0.0000000e+00 -4.4408921e-16  0.0000000e+00]
14:37:22 - atmodeller.core      - DEBUG     - residual_mass = [4.78044480e-06 2.01684767e-05]
14:37:22 - atmodeller.core      - DEBUG     - residual = [ 0.00000000e+00 -4.44089210e-16  0.00000000e+00  4.78044480e-06
  2.01684767e-05]
14:37:22 - atmodeller.core      - DEBUG     - residual_reaction = [0.0000000e+00 4.4408921e-16 0.0000000e+00]
14:37:22 - atmodeller.core      - DEBUG     - residual_mass = [8.90974233e-08 4.18447901e-07]
14:37:22 - atmodeller.core      - DEBUG     - residual = [0.00000000e+00 4.44089210e-16 0.00000000e+00 8.90974233e-08
 4.18447901e-07]
14:37:22 - atmodeller.core      - DEBUG     - residual_reaction = [0. 0. 0.]
14:37:22 - atmodeller.core      - DEBUG     - residual_mass = [2.05857168e-10 9.82318766e-10]
14:37:22 - atmodeller.core      - DEBUG     - residual = [0.00000000e+00 0.00000000e+00 0.00000000e+00 2.05857168e-10
 9.82318766e-10]
14:37:22

{'CO': 62.30341145750454,
 'H2': 0.5547795429001989,
 'O2': 3.9099946023954954e-08,
 'CO2': 9.46718162781583,
 'H2O': 0.378344754346716}

### 4. Including more species such as CH4

It is straightforward to add more species to the system, although they must have their formation energies and masses already specified in the code.

In [18]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CH4', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules

14:37:49 - atmodeller.core      - INFO      - Creating a molecule: H2O
14:37:49 - atmodeller.core      - INFO      - Creating a molecule: H2
14:37:49 - atmodeller.core      - INFO      - Creating a molecule: O2
14:37:49 - atmodeller.core      - INFO      - Creating a molecule: CO
14:37:49 - atmodeller.core      - INFO      - Creating a molecule: CO2
14:37:49 - atmodeller.core      - INFO      - Creating a molecule: CH4


Element count: {'H': 2, 'O': 1}
Element count: {'H': 2}
Element count: {'O': 2}
Element count: {'C': 1, 'O': 1}
Element count: {'C': 1, 'O': 2}
Element count: {'C': 1, 'H': 4}


[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x120a5ea50>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, formation_constants=(0.05817258823529415, -251.80119852941178), molar_mass=0.018015200000000002),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x10b3e3590>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, formation_constants=(0, 0), molar_mass=0.0020158),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x10b52f510>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, formation_constants=(0, 0), molar_mass=0.0319988),
 Molecule(name='CO', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x120a52790>, solid_melt_distribution_coefficient=0, elements={'C': 1, 'O': 1}, element_masses={'C': 0.0120107, 'O': 0.0159994}, fo

We define a mixture of mass and oxygen fugacity constraints as before and solve the system. CH4 is not prevalent at 2000 K so the results are almost identical to those without CH4 presented above.

In [20]:
number_of_earth_oceans: float = 1
# C/H ratio by mass.
ch_ratio: float = 1

mass_H: float = number_of_earth_oceans * OCEAN_MOLES * MolarMasses().H2
mass_C: float = ch_ratio * mass_H

constraints: list[SystemConstraint] = [
    SystemConstraint(species="H", value=mass_H, field="mass"),
    SystemConstraint(species="C", value=mass_C, field="mass"),
]

interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules)
interior_atmosphere.solve(constraints, fo2_constraint=True)
interior_atmosphere.pressures_dict

14:37:57 - atmodeller.core      - INFO      - Creating a new planet
14:37:57 - atmodeller.core      - INFO      - Mantle mass (kg) = 4.208261222595111e+24
14:37:57 - atmodeller.core      - INFO      - Mantle melt fraction = 1.0
14:37:57 - atmodeller.core      - INFO      - Core mass fraction = 0.295334691460966
14:37:57 - atmodeller.core      - INFO      - Planetary radius (m) = 6371000.0
14:37:57 - atmodeller.core      - INFO      - Planetary mass (kg) = 5.972e+24
14:37:57 - atmodeller.core      - INFO      - Surface temperature (K) = 2000.000000
14:37:57 - atmodeller.core      - INFO      - Surface gravity (m/s^2) = 9.819973426224687
14:37:57 - atmodeller.core      - INFO      - Oxygen fugacity model (mantle) = IronWustiteBufferOneill
14:37:57 - atmodeller.core      - INFO      - Oxygen fugacity log10 shift = 0.000000
14:37:57 - atmodeller.core      - INFO      - Creating a new interior-atmosphere system
14:37:57 - atmodeller.core      - INFO      - Molecules = ['CO', 'H2', 'O2', 'CO

['H', 'C', 'O']


14:37:57 - atmodeller.core      - DEBUG     - residual_reaction = [ 0.00267714 -0.03639737 -0.00617217 -0.02041079]
14:37:57 - atmodeller.core      - DEBUG     - residual_mass = [ 0.03184529 -0.98628802]
14:37:57 - atmodeller.core      - DEBUG     - residual = [ 0.00267714 -0.03639737 -0.00617217 -0.02041079  0.03184529 -0.98628802]
14:37:57 - atmodeller.core      - DEBUG     - residual_reaction = [ 0.00093969 -0.0305826   0.01132475 -0.01751651]
14:37:57 - atmodeller.core      - DEBUG     - residual_mass = [2.39295445e+08 2.75593864e+16]
14:37:57 - atmodeller.core      - DEBUG     - residual = [ 9.39688414e-04 -3.05825972e-02  1.13247455e-02 -1.75165071e-02
  2.39295445e+08  2.75593864e+16]
14:37:57 - atmodeller.core      - DEBUG     - residual_reaction = [ 6.96790181e-10  3.63255381e-09  6.82146073e-09 -5.09690068e-11]
14:37:57 - atmodeller.core      - DEBUG     - residual_mass = [ 0.00315842 -0.98615992]
14:37:57 - atmodeller.core      - DEBUG     - residual = [ 6.96790181e-10  3.63

{'CO': 62.303409879195826,
 'H2': 0.5547795364480278,
 'O2': 3.9099946023954954e-08,
 'CO2': 9.467181387987301,
 'H2O': 0.37834474994650863,
 'CH4': 1.283294412915622e-06}